### Imports 

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from tqdm.notebook import tqdm
import pandas as pd
import pingouin as pg
from neuropy import plotting
import numpy as np
import subjects

### Single session

In [ ]:
import numpy as np
import subjects
from neuropy.analyses import ExplainedVariance
from neuropy.analyses.neurons_stability import isi_stability, firing_rate_stability

sess = subjects.sdrol.ratN_2019_10_19[0]
neurons = sess.neurons_stable.get_neuron_type('pyr')


# #---- isi stability --------
# isi_bool = isi_stability(neurons)
# stable_cells_indx = np.where(isi_bool==True)[0]
# stable_neurons = neurons[stable_cells_indx]

wave_similarity = neurons.get_waveform_similarity()
pairs_bool = wave_similarity < 0.8  # only pairs which are least similar
# shank_ids = neurons.shank_ids
# pairs_bool = shank_ids.reshape(-1, 1) - shank_ids.reshape(1, -1) #cross_shanks only


pre, maze, post = (
    sess.paradigm["pre"].flatten(),
    sess.paradigm["maze"].flatten(),
    sess.paradigm["post"].flatten(),
)

expvar = ExplainedVariance(
    neurons=neurons,
    template=maze,
    matching=[post[0],post[1]-3600],
    control=pre,
    window=900,
    slideby=300,
    pairs_bool=pairs_bool,
    ignore_epochs=sess.artifact,
)
print(expvar.n_pairs)

In [ ]:
sess.paradigm.durations/3600

In [ ]:
_,ax = plt.subplots()

# plotting.plot_raster(neurons=neurons,ax=ax)
binspk = neurons.get_binned_spiketrains(bin_size=300)
corrmat = np.corrcoef(binspk.spike_counts.T)
np.fill_diagonal(corrmat,0)

# ax.plot(binspk.spike_counts.mean(axis=0))
ax.imshow(corrmat,cmap='jet')

In [ ]:
from neuropy.plotting import plot_hypnogram,Fig

fig=Fig(grid=(5,6))

ax = fig.subplot(fig.gs[1:4])
expvar.plot(ax=ax,t_start=post[0],show_ignore_epochs=False)
ax.set_xlim(left=0)
# ax.set_ylim(top=0.14)


# ax = fig.subplot(fig.gs[0])
# plot_hypnogram(sess.brainstates,ax=ax)
# ax.set_xlim(left = post[0])
